In [1]:
from main import *
import importlib

In [2]:
messages = [
        {
        "role": "system",
        "content": "You are an expert computational materials scientist. \nYou are trying to develop an experiment to crystallise carbon into diamond.\nPredict appropriate code for a melt-quench simulation of this system. \nTry NOT to use an extremely large number of timesteps.\nThe format of the code should be a LAMMPS input file like:\n# Phase PHASE:\nfix integrate all nvt temp T_INITIAL T_FINAL 0.1\nrun TIMESTEPS\nunfix integrate\n"
        },
        {
        "role": "user",
        "content": "RESULT:\nSOAP similarity to diamond is 0.1\nSOAP similarity to graphite is 0.9\n\nSCRIPT:"
        },
        {
        "role": "assistant",
        "content": "# Phase 1: \nfix integrate all nvt temp 9000 9000 0.1\nrun 100000\nunfix integrate\n\n# Phase 2: \nfix integrate all nvt temp 9000 3000 0.1\nrun 100000 \nunfix integrate\n\n# Phase 3: \nfix integrate all nvt temp 3000 500 0.1\nrun 100000 \nunfix integrate\n\n# Phase 4:\nfix integrate all nvt temp 500 500 0.1\nrun 100000 \nunfix integrate\n\n# Phase 5:\nfix integrate all nvt temp 500 300 0.1\nrun 100000\nunfix integrate"
        },
        {
        "role": "user",
        "content": "RESULT:\nSOAP similarity to diamond is 1.0\nSOAP similarity to graphite is 0.0\n\nSCRIPT:"
        },
    ]


In [3]:
code, messages, response = query_llm(messages)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "# Phase 1: \nfix integrate all nvt temp 5000 5000 0.1\nrun 50000\nunfix integrate\n\n# Phase 2: \nfix integrate all nvt temp 5000 3000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 3: \nfix integrate all nvt temp 3000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 4:\nfix integrate all nvt temp 1000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 5:\nfix integrate all nvt temp 1000 300 0.1\nrun 50000\nunfix integrate",
        "role": "assistant"
      }
    }
  ],
  "created": 1698436785,
  "id": "chatcmpl-8ENHdynfQ0HCYct3ZkTIMOh5KdiuM",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 156,
    "prompt_tokens": 313,
    "total_tokens": 469
  }
}
[{'role': 'system', 'content': 'You are an expert computational materials scientist. \nYou are trying to develop an experiment to crystallise carbon into diamond.\nPredict appropriate code

In [4]:
code

'# Phase 1: \nfix integrate all nvt temp 5000 5000 0.1\nrun 50000\nunfix integrate\n\n# Phase 2: \nfix integrate all nvt temp 5000 3000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 3: \nfix integrate all nvt temp 3000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 4:\nfix integrate all nvt temp 1000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 5:\nfix integrate all nvt temp 1000 300 0.1\nrun 50000\nunfix integrate'

In [5]:
s = format_experiment()
s

'\nlog /home/jbutch/Projects/Thesis/llms-for-lammps/log.dat append\n\nunits metal     # mass = g/mole, distance = Å, time = ps, energy = eV, force = eV/Å\natom_style atomic\nread_data /home/jbutch/Projects/Thesis/llms-for-lammps/structures/example.extxyz\nmass * 12.011   # all atoms are Carbon and therefore have mass ~12\n\npair_style quip\npair_coeff * * /home/jbutch/Projects/Thesis/llms-for-lammps/data/carbon.xml "" 6\n\nneigh_modify every 1 delay 0 check yes\ntimestep 0.001\n\nvariable nAtoms equal atoms\n\nfix removeMomentum all momentum 1 linear 1 1 1\n\ncompute T all temp\nvariable P equal press\nvariable v equal vol\nvariable PE_Atom equal c_PE/v_nAtoms\n\nfix TempAve all ave/time 100000 1e-06 100 c_T\nfix PressAve all ave/time 100000 1e-06 100 v_P\nfix vAve all ave/time 100000 1e-06 100 v_v\nfix PEAve_Atom all ave/time 100000 1e-06 100 v_PE_Atom\n\ncompute PE all pe pair\ncompute MSD all msd\n\nthermo_style custom step cpu temp f_TempAve press f_PressAve f_PEAve_Atom vol f_vAve

In [9]:
full_code = s + code
f_in="/home/jbutch/Projects/Thesis/llms-for-lammps/lammps/test.in"
with open(f_in, "w") as f:
    f.write(full_code)

print(full_code)


log /home/jbutch/Projects/Thesis/llms-for-lammps/log.dat append

units metal     # mass = g/mole, distance = Å, time = ps, energy = eV, force = eV/Å
atom_style atomic
read_data /home/jbutch/Projects/Thesis/llms-for-lammps/structures/example.extxyz
mass * 12.011   # all atoms are Carbon and therefore have mass ~12

pair_style quip
pair_coeff * * /home/jbutch/Projects/Thesis/llms-for-lammps/data/carbon.xml "" 6

neigh_modify every 1 delay 0 check yes
timestep 0.001

variable nAtoms equal atoms

fix removeMomentum all momentum 1 linear 1 1 1

compute T all temp
variable P equal press
variable v equal vol
variable PE_Atom equal c_PE/v_nAtoms

fix TempAve all ave/time 100000 1e-06 100 c_T
fix PressAve all ave/time 100000 1e-06 100 v_P
fix vAve all ave/time 100000 1e-06 100 v_v
fix PEAve_Atom all ave/time 100000 1e-06 100 v_PE_Atom

compute PE all pe pair
compute MSD all msd

thermo_style custom step cpu temp f_TempAve press f_PressAve f_PEAve_Atom vol f_vAve c_MSD[4]
thermo_modify flush ye

In [11]:
import subprocess
import time
import re

# Submit the job and capture the output
result = subprocess.run(['qsub', 'submit.sh'], stdout=subprocess.PIPE, text=True)
output = result.stdout
print('Submission output:', output)

# Extract the job ID from the output
job_id_match = re.search(r'(\d+)', output)
if job_id_match is None:
    print('Failed to submit job or parse job ID')
    exit(1)

job_id = job_id_match.group(1)
print('Job ID:', job_id)

# Check the job status repeatedly until it's no longer in the queue
while True:
    # Query the job status
    result = subprocess.run(['qstat', '-j', job_id], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # If 'qstat' returns an error, it likely means the job is no longer in the queue
    if result.returncode != 0:
        print('Job completed or failed')
        break
    
    # Optional: Print the job status
    print('Job is still in the queue...')
    
    # Wait for a short duration before checking again
    time.sleep(10)  # Wait for 10 seconds


FileNotFoundError: [Errno 2] No such file or directory: 'qsub'